In [2]:
from llm.chatbot import chat_model
from langchain.schema.messages import HumanMessage, SystemMessage

In [4]:
messages = [
     SystemMessage(
         content="""You're an assistant knowledgeable about
         healthcare. Only answer healthcare-related questions."""
     ),
     HumanMessage(content="What is Medicaid managed care?"),
]
chat_model.invoke(messages).content

"Medicaid managed care is a system where state Medicaid programs contract with managed care organizations (MCOs) to provide healthcare services to Medicaid enrollees. In this model, the MCOs assume the risk and responsibility for providing a defined set of healthcare services to enrolled Medicaid beneficiaries, often in exchange for a fixed monthly payment per member.\n\nUnder Medicaid managed care, enrollees typically choose a primary care provider (PCP) from a network of participating healthcare providers who coordinate their care. The MCOs manage the delivery of healthcare services, including authorizing and paying for medically necessary services, coordinating specialty care, and emphasizing preventive care.\n\nMedicaid managed care plans can vary in their specific designs and may include models such as:\n\n1. Health Maintenance Organizations (HMOs): Enrollees typically need to seek care from providers within the HMO's network, except in emergencies.\n\n2. Preferred Provider Organi

In [11]:
from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)

review_system_template_str = """Your job is to use patient
reviews to answer questions about their experience at a
hospital. Use the following context to answer questions.
Be as detailed as possible, but don't make up any information
that's not from the context. If you don't know an answer, say
you don't know.

{context}
"""

review_system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["context"], template=review_system_template_str
    )
)

review_human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["question"], template="{question}"
    )
)

messages = [review_system_prompt, review_human_prompt]
review_prompt_template = ChatPromptTemplate(
    input_variables=["context", "question"],
    messages=messages,
)
context = "I had a great stay!"
question = "Did anyone have a positive experience?"

review_prompt_template.format_messages(context=context, question=question)

[SystemMessage(content="Your job is to use patient\nreviews to answer questions about their experience at a\nhospital. Use the following context to answer questions.\nBe as detailed as possible, but don't make up any information\nthat's not from the context. If you don't know an answer, say\nyou don't know.\n\nI had a great stay!\n"),
 HumanMessage(content='Did anyone have a positive experience?')]

In [17]:
import os
import pandas as pd
from dotenv import load_dotenv, find_dotenv


load_dotenv(find_dotenv())
COHERE_API_KEY = os.environ["COHERE_API_KEY"]
REVIEWS_CSV_PATH = "data/reviews.csv"
REVIEWS_CHROMA_PATH = "llm/chroma_data"


# loader = CSVLoader(file_path=REVIEWS_CSV_PATH, source_column="review")
# reviews = loader.load()

# reviews_vector_db = Chroma.from_documents(
#     reviews, CohereEmbeddings(), persist_directory=REVIEWS_CHROMA_PATH
# )

## Concat DataFrame

In [18]:
data = pd.read_csv("data/visits.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9998 entries, 0 to 9997
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   patient_id             9998 non-null   int64  
 1   date_of_admission      9998 non-null   object 
 2   billing_amount         9998 non-null   float64
 3   room_number            9998 non-null   int64  
 4   admission_type         9998 non-null   object 
 5   discharge_date         9498 non-null   object 
 6   test_results           9998 non-null   object 
 7   visit_id               9998 non-null   int64  
 8   physician_id           9998 non-null   int64  
 9   payer_id               9998 non-null   int64  
 10  hospital_id            9998 non-null   int64  
 11  chief_complaint        3000 non-null   object 
 12  treatment_description  3000 non-null   object 
 13  primary_diagnosis      3000 non-null   object 
 14  visit_status           9998 non-null   object 
dtypes: f

In [16]:
# data = data[:20]
# data.to_csv("data/reviews_20.csv", index=False)

In [19]:
data.head()

,patient_id,date_of_admission,billing_amount,room_number,admission_type,discharge_date,test_results,visit_id,physician_id,payer_id,hospital_id,chief_complaint,treatment_description,primary_diagnosis,visit_status
0,0,2022-11-17,37490.983364,146,Elective,2022-12-01,Inconclusive,0,102,1,0,NaN,NaN,NaN,DISCHARGED
1,1,2023-06-01,47304.064845,404,Emergency,NaN,Normal,1,435,4,5,NaN,NaN,NaN,OPEN
2,2,2019-01-09,36874.896997,292,Emergency,2019-02-08,Normal,2,348,2,6,NaN,NaN,NaN,DISCHARGED
3,3,2020-05-02,23303.322092,480,Urgent,2020-05-03,Abnormal,3,270,4,15,NaN,NaN,NaN,DISCHARGED
4,4,2021-07-09,18086.344184,477,Urgent,2021-08-02,Normal,4,106,2,29,Persistent cough and shortness of breath,Prescribed a combination of inhaled bronchodil...,"J45.909 - Unspecified asthma, uncomplicated",DISCHARGED


We will use data visit to build relationship every data using id, something like visit_id, physician_id, payer_id, hospital_id and patient_id and review_id from data reviews.csv

## MultiVector

In [48]:
from langchain.storage import InMemoryByteStore
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import Chroma
from langchain_cohere import CohereEmbeddings
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [49]:
loaders = [
    CSVLoader("data/summary/hospital_summary.csv", source_column="summary"),
    CSVLoader("data/summary/patient_summary.csv", source_column="summary"),
    CSVLoader("data/summary/payer_summary.csv", source_column="summary"),
    CSVLoader("data/summary/physician_summary.csv", source_column="summary"),
    CSVLoader("data/summary/visit_summary.csv", source_column="summary"),
    CSVLoader("data/reviews.csv", source_column="review"),
]

docs = []
for loader in loaders:
    docs.extend(loader.load())
text_splitter = RecursiveCharacterTextSplitter()
docs = text_splitter.split_documents(docs)

In [51]:
# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="full_documents", embedding_function=CohereEmbeddings(cohere_api_key=COHERE_API_KEY)
)
# The storage layer for the parent documents
store = InMemoryByteStore()
id_key = "doc_id"
# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)
import uuid

doc_ids = [str(uuid.uuid4()) for _ in docs]

In [56]:
retriever.vectorstore.add_documents(docs)

In [ ]:
# Vectorstore alone retrieves the small chunks
retriever.vectorstore.similarity_search("justice breyer")[0]